In [1]:
import cvxpy as cp
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

import time
from joblib import Parallel, delayed
from os import cpu_count

import utils
import spectral_nti as snti

SEED = 0
N_CPUS = cpu_count()
np.random.seed(SEED)

## Auxiliary functions

In [2]:
def est_graph(id, models, MM, X):
    N = X.shape[0]
    A_hat = np.zeros((N, N, len(models), len(MM)))
    lamd_hat = np.zeros((N,len(models), len(MM)))
    t = time.time()
    for i,M in enumerate(MM):
        C_hat = X[:,:M]@X[:,:M].T/M

        for j, model in enumerate(models):
            L_hat, _ = \
                snti.SGL_MM(C_hat, model['gs'], model['bounds'],
                            model['cs'], model['regs'], max_iters=iters)

            A_hat[:,:,j,i] = np.diag(np.diag(L_hat)) - L_hat
            lamd_hat[:,j,i], _ = np.linalg.eigh(L_hat)
    t = time.time() - t
    print('Cov-{} - Time(sec): {:.3}'.format(id,  t))
    return A_hat, lamd_hat


def est_errs(A_hat, A):
    L = np.diag(np.sum(A,0)) - A
    norm_A = np.linalg.norm(A,'fro')
    norm_L = np.linalg.norm(L,'fro')
    
    err_A = np.zeros(A_hat.shape[2:])
    err_L = np.zeros(A_hat.shape[2:])
    for k in range(A_hat.shape[-1]):
        for i in range(A_hat.shape[-2]):
            for j in range(A_hat.shape[-3]):
                L_hat = np.diag(np.sum(A_hat[:,:,j,i,k],0)) - A_hat[:,:,j,i,k]

                err_A[j,i,k] = np.linalg.norm(A-A_hat[:,:,j,i,k],'fro')**2/norm_A**2
                err_L[j,i,k] = np.linalg.norm(L-L_hat,'fro')**2/norm_L**2
    return err_A, err_L


def est_errs2(A_hat, A):
    L = np.diag(np.sum(A,0)) - A
    norm_A = np.linalg.norm(A,'fro')
    norm_L = np.linalg.norm(L,'fro')

    err_A = np.zeros(A_hat.shape[2:])
    err_L = np.zeros(A_hat.shape[2:])
    for k in range(A_hat.shape[-1]):
        for i in range(A_hat.shape[-2]):
            for j in range(A_hat.shape[-3]):
                L_hat = np.diag(np.sum(A_hat[:,:,j,i,k],0)) - A_hat[:,:,j,i,k]
                if np.all((A_hat[:,:,j,i,k] == 0)):
                    print('WARNING: A_hat is 0.')
                    norm_A_hat = 1
                    norm_lamb_hat = 1
                else:
                    norm_A_hat = np.linalg.norm(A_hat[:,:,j,i,k], 'fro')
                    norm_L_hat = np.linalg.norm(L_hat, 'fro')

                A_hat_norm = A_hat[:,:,j,i,k]/norm_A_hat
                L_hat_norm = L_hat/norm_L_hat
                err_A[j,i,k] = np.linalg.norm(A/norm_A-A_hat_norm,'fro')**2
                err_L[j,i,k] = np.linalg.norm(L/norm_L-L_hat_norm,'fro')**2
    return err_A, err_L


def plot_err(MM, models, err, ylab, semlogy=True):
    plt.figure()
    for i, model in enumerate(models):
        if semlogy:
            plt.semilogy(MM, err[i,:], model['fmt'], label=model['name'],
                        linewidth=2, markersize=12)
        else:
            plt.plot(MM, err[i,:], model['fmt'], label=model['name'],
                     linewidth=2, markersize=12)
        plt.grid(True)
        plt.xlabel('Number of samples')
        plt.ylabel(ylab)
        plt.legend()
        plt.tight_layout()


def print_err(MM, models, err):
    mean_err = np.mean(err, 2)
    std = np.std(err, 2)
    for i, M in enumerate(MM):
        print('M:', M)
        for j, model in enumerate(models):
            print('\t{}: mean err: {:.6f} - std: {:.6f}'.
                  format(model['name'], mean_err[j,i], std[j,i]))

## Create graphs

In [4]:
# Ref grap
n01 = 15
n02 = 10
N0 = n01*n02
A0 = nx.to_numpy_array(nx.grid_2d_graph(n01, n02))
L0 = np.diag(np.sum(A0, 0)) - A0
lambdas0, _ = np.linalg.eigh(L0)

# Target graph
n1 = 20
n2 = 10
N = n1*n2
A = nx.to_numpy_array(nx.grid_2d_graph(n1, n2))
L = np.diag(np.sum(A, 0)) - A
lambdas, V = np.linalg.eigh(L)

norm_A = np.linalg.norm(A,'fro')
norm_lamb = np.linalg.norm(lambdas)

print('Norm of A:', norm_A)
print('Norm of lambdas:', norm_lamb)

# Plot graphs and spectrum distribution
plt.figure()
plt.subplot(1,2,1)
plt.imshow(A0)
plt.title('Ref A, N0: ' + str(N0))
plt.subplot(1,2,2)
plt.imshow(A)
plt.title('True A, N: ' + str(N))
    
figs, axs = plt.subplots(1, 2, sharey=True)
axs[0].hist(lambdas0, density=True)
axs[0].set_title('Ref Lambdas, N0: ' + str(N0))
axs[1].hist(lambdas, density=True)
axs[1].set_title('True Lambdas, N: ' + str(N))

Norm of A: 27.202941017470888
Norm of lambdas: 59.39696961966998


## Test influence of tightening constraints

In [18]:
n_covs = 100
MM = [500, 750, 1000]
iters = 200

GS = [
    lambda a, b : cp.sum(a)/b,
    lambda a, b : cp.sum(a**2)/b,
    lambda a, b : cp.sum(cp.exp(-a))/b,
    lambda a, b : cp.sum(cp.sqrt(a))/b,
    lambda a, b : cp.sum(.25*a**2-.75*a)/b,
]
BOUNDS = [
    lambda lamd, lamd_t, b : -2/b*lamd_t.T@lamd,
    lambda lamd, lamd_t, b : 1/b*cp.exp(-lamd_t).T@lamd,
    lambda lamd, lamd_t, b : cp.sum(lamd/cp.sqrt(lamd_t))/(2*b),
    lambda lamd, lamd_t, b: 1/b*(0.75-2*0.25*lamd_t).T@lamd,
]

deltas = [.04, .27, .003, .02, 0.05]
cs = utils.compute_cs(GS, lambdas0, lambdas)

models = [
          {'name': 'Sq', 'gs': GS[1], 'bounds': [], 'cs': cs[1], 'fmt': 'o-',
           'regs': {'alpha': 0, 'beta': 1.5, 'gamma': 0, 'deltas': deltas[1]}},

          {'name': 'Sq-T', 'gs': GS[1], 'bounds': BOUNDS[0], 'cs': cs[1], 'fmt': 'o--',
           'regs': {'alpha': .0, 'beta': 1.25, 'gamma': 1000, 'deltas': deltas[1]}},

          {'name': 'Heat', 'gs': GS[2], 'bounds': [], 'cs': cs[2], 'fmt': 'x-',
           'regs': {'alpha': .001, 'beta': .5, 'gamma': 0, 'deltas': deltas[2]}},

          {'name': 'Heat-T', 'gs': GS[2], 'bounds': BOUNDS[1], 'cs': cs[2], 'fmt': 'x--',
           'regs': {'alpha': .001, 'beta': .4, 'gamma': 100, 'deltas': deltas[2]}},

          {'name': 'Sqrt', 'gs': GS[3], 'bounds': [], 'cs': cs[3], 'fmt': 'v-',
           'regs': {'alpha': .01, 'beta': .5, 'gamma': 0, 'deltas': deltas[3]}},

          {'name': 'Sqrt-T', 'gs': GS[3], 'bounds': BOUNDS[2], 'cs': cs[3], 'fmt': 'v--',
           'regs': {'alpha': .01, 'beta': .5, 'gamma': 25, 'deltas': deltas[3]}},

          {'name': 'Poly', 'gs': GS[4], 'bounds': [], 'cs': cs[4], 'fmt': 's-',
           'regs': {'alpha': 0, 'beta': 1.25, 'gamma': 0, 'deltas': deltas[4]}},

          {'name': 'Poly-T', 'gs': GS[4], 'bounds': BOUNDS[3], 'cs': cs[4], 'fmt': 's--',
           'regs': {'alpha': 0, 'beta': 1, 'gamma': 1000, 'deltas': deltas[4]}},
]

	c-0: c: 3.700	c0: 3.667	err: 0.033333	err norm: 0.009091
	c-1: c: 17.640	c0: 17.387	err: 0.253333	err norm: 0.014571
	c-2: c: 0.114	c0: 0.115	err: -0.001259	err norm: -0.010911
	c-3: c: 1.828	c0: 1.817	err: 0.010217	err norm: 0.005622
	c-4: c: 1.635	c0: 1.597	err: 0.038333	err norm: 0.024008


In [19]:
# Create signals
lambdas_aux = np.concatenate(([0], 1/np.sqrt(lambdas[1:])))
C_inv_sqrt = V@np.diag(lambdas_aux)@V.T
X = C_inv_sqrt@np.random.randn(n_covs, N, MM[-1])

# Estimate graph
total_t = time.time()
As_hat = np.zeros((N, N, len(models), len(MM), n_covs))
lamds_hat = np.zeros((N, len(models), len(MM), n_covs)) 

print('N_CPUS:', N_CPUS)
pool = Parallel(n_jobs=N_CPUS)
resps = pool(delayed(est_graph)(i, models, MM, X[i,:,:]) for i in range(n_covs))
for i, resp in enumerate(resps):
    As_hat[:,:,:,:,i], lamds_hat[:,:,:,i] = resp

total_t = time.time() - total_t
print('-----', total_t/60, ' mins -----')

N_CPUS: 12


In [ ]:
err_A, err_L = est_errs(As_hat, A)
print_err(MM, models, err_L)

# Plot error 
mean_err = np.mean(err_A, 2)
mean_err_L = np.mean(err_L, 2)
plot_err(MM, models, mean_err, 'Mean Err1 A')
plot_err(MM, models, mean_err_L, 'Mean Err1 L')

In [ ]:
err_A, err_L = est_errs2(As_hat, A)
print_err(MM, models, err_L)

# Plot error 
mean_err = np.mean(err_A, 2)
mean_err_L = np.mean(err_L, 2)
plot_err(MM, models, mean_err, 'Mean Err1 A')
plot_err(MM, models, mean_err_L, 'Mean Err1 L')

In [ ]:
# Save data
models_aux = [{'name': model['name'], 'fmt': model['fmt'], 'regs': model['regs']}
                for model in models]
data = {
    'A': A,
    'lambdas': lambdas,
    'As_hat': As_hat,
    'lamds_hat': lamds_hat,
    'MM': MM,
    'models': models_aux
}
# np.save('results/tight_constraints/tight_constraints', data)